In [1]:
# Denpendancies
import os
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Load tickers from Database
def get_tickers(database='Database'):
    db = os.path.join(os.getcwd(), database)
    if os.path.exists(db):
        tickers = [ld.split('.')[0] for ld in os.listdir(db)]
        return sorted(tickers)
    print('Database could not be found.')

def batching(data, period):
    batch = len(data) // period
    max_length = batch * period
    return np.array(data)[-max_length:].reshape(-1, period)

def regroup(data, period):
    base = batching(data['Open'], period)[:, 0]
    high = batching(data['High'], period).max(1)
    low = batching(data['Low'], period).min(1)
    close = batching(data['Close'], period)[:, -1]
    return pd.DataFrame(np.stack([base, high, low, close], axis=1), 
                        columns=['Open', 'High', 'Low', 'Close'])

def get_pep(g, time=(7.5 / .5)):
    m = g > 0
    pos = g[m].mean()
    neg = np.abs(g[~m].mean()) if not np.isnan(g[~m].mean()) else 0
    _rang = pos + neg
    return _rang / time

In [52]:
ticker = DataLoader('AAPL').get_data('2018-01-01', '2020-12-31')

In [53]:
# Calculate Peps
hc = (ticker['High'].shift(-1) / ticker['Close'] - 1).dropna()
cc = (ticker['Close'].shift(-1) / ticker['Close'] - 1).dropna()
oc = (ticker['Open'].shift(-1) / ticker['Close'] - 1).dropna()
hl = (ticker['High'] / ticker['Low'] - 1).dropna()

def print_report(g):
    m = g > 0
    plen = len(g[m]) / len(g)
    pM = g[m].mean()
    nlen = len(g[~m]) / len(g)
    nM = g[~m].mean() if not np.isnan(g[~m].mean()) else 0
    _rang = pM + np.abs(nM)
    pep = _rang / (7.5 / .5)
    print('{:.2f}::{:.4f} | {:.2f}::{:>7.4f} | {:.4f}'.format(
        plen, pM, nlen, nM, pep))
    return {'P PCT': plen, 
            'P Mean': pM, 
            'N PCT': nlen, 
            'N Mean': nM, 
            'Pep': pep}

oc_dic = print_report(oc)
cc_dic = print_report(cc)
hc_dic = print_report(hc)
hl_dic = print_report(hl)

0.57::0.0084 | 0.43::-0.0097 | 0.0012
0.54::0.0154 | 0.46::-0.0146 | 0.0020
0.86::0.0160 | 0.14::-0.0099 | 0.0017
1.00::0.0227 | 0.00:: 0.0000 | 0.0015


In [56]:
pep = get_pep(hl)
print(pep)
ticker.Close.iloc[-1] * pep

0.0015165436619617079


0.20123018153878633

In [51]:
def back_test(ticker, pep, s=6, hs=3, os=2):
    holding = False
    max_shares = 1000
    #
    base = 0
    shares = 0
    #
    capital = 1000
    base_capital = capital
    #
    sell = 1 + (s * pep)
    for _, prices in ticker.iterrows():
        if not holding:
            base = prices.Close
            shares = capital // base
            shares = max_shares if shares > max_shares else shares
            #
            min_sell = base * sell
            stop_price = base * (1 + (-os * pep))
            #
            holding = True
        else:
            if prices.Open < stop_price:
                gain = prices.Open - base
            elif prices.High > min_sell:
                gain = (prices.High * (1 + (-hs * pep))) - base
            else:
                gain = prices.Close - base

            capital += np.round(gain * shares, 2)
            # 
            base = 0
            shares = 0
            #
            holding = False

    return capital / base_capital - 1

In [7]:
back_test(ticker, pep)

7.188399999999996

In [8]:
tickers = get_tickers()

In [46]:
dic = {}
for ticker in tickers:
    data = DataLoader(ticker).get_data('2018-01-01', '2020-12-31')
    data = regroup(data, 3)
    g = (data['High'] / data['Low'] - 1).dropna()
    pep = get_pep(g, time=(7.5 / (15 / 60)))
    above_pep = (g > pep).mean()
    data = DataLoader(ticker).get_data('2021-01-01', '2021-3-31')
    data = regroup(data, 3)
    cap_gain = back_test(data, pep)
    dic[ticker] = {'PEP': pep, 'Above Pep': above_pep, 'Cap Gain': cap_gain}

In [47]:
df = pd.DataFrame.from_dict(dic, orient='index')

In [48]:
df.sort_values(by='Cap Gain', ascending=False)

,PEP,Above Pep,Cap Gain
AMC,0.004576,1.000000,10.62098
OCGN,0.009159,1.000000,6.14537
HOFV,0.002595,0.601695,4.62073
RUBY,0.005251,1.000000,4.52963
ASXC,0.006457,1.000000,4.05436
...,...,...,...
HYRE,0.006539,1.000000,-0.20569
MNMD,0.029770,0.801587,-0.20909
CLSK,0.012194,0.809524,-0.24431
FOLD,0.003225,1.000000,-0.32184


In [49]:
mask = (df['Above Pep'] > .9) & (df['Cap Gain'] > 5)
df[mask].sort_values(by='Cap Gain', ascending=False)

,PEP,Above Pep,Cap Gain
AMC,0.004576,1.0,10.62098
OCGN,0.009159,1.0,6.14537


In [50]:
gain = 0
for ticker in df[mask].index:
    data = DataLoader(ticker).get_data('2021-04-01', '2021-12-31')
    data = regroup(data, 3)
    pep = df.loc[ticker, 'PEP']
    cap_gain = back_test(data, pep)
    print('{:>6} | {:>8.4f} | {:>8.4f}'.format(ticker, pep, cap_gain))
    gain += cap_gain
print('Total gain for {:>4} stocks: {:>8.2f}%'.format(mask.sum(), gain / mask.sum() * 100))

   AMC |   0.0046 |   9.4182
  OCGN |   0.0092 |   1.4279
Total gain for    2 stocks:   542.31%
